### Competency questions

In [1]:
import rdflib
from rdflib import Graph
from rdflib.namespace import Namespace
from rdflib.namespace import SKOS, RDF

In [2]:
culco = Namespace("http://example.nl/ontology#")
skosxl = Namespace("http://www.w3.org/2008/05/skos-xl#")
dcterms = Namespace("http://purl.org/dc/terms/")

In [3]:
wm = Graph()

In [4]:
wm.parse("https://raw.githubusercontent.com/cultural-ai/wordsmatter/main/glossary.ttl", format="turtle")

<Graph identifier=N87cc8f7735c84b0bac6913bdb0cc6fad (<class 'rdflib.graph.Graph'>)>

In [5]:
len(wm)

2492

#### 1. Which entries have references to other entries?

In [7]:
q_1 = wm.query(
    """SELECT ?CI ?cont_issue (GROUP_CONCAT(?CI_2;SEPARATOR=",") AS ?CI_2_list)
        
        {
            ?CI dcterms:references ?CI_2 .
            ?CI dcterms:title ?cont_issue .
        }
         GROUP BY ?CI
         """)

for row in q_1:
    print(f"{row.cont_issue}({row.CI.split('#')[-1]}): {[e.split('#')[-1] for e in row.CI_2_list.split(',')]}")

Zwart(149_nl): ['133_nl']
Descent(101): ['91', '136']
Halfbloed(110_nl): ['140_nl']
Headhunter(111): ['132']
Wit(147_nl): ['100_nl']
Gay(108_nl): ['113_nl']
Negro(129): ['135', '98']
Aboriginal(94_nl): ['116_nl', '105_nl']
Blank(95): ['144']
Mulatto(127): ['110']
Race(135): ['98', '129']
Mongool(130_nl): ['124_nl', '133_nl']
Indisch(117_nl): ['118_nl']
Blank(100_nl): ['149_nl', '147_nl', '109_nl']
Bosneger(102_nl): ['133_nl', '127_nl']
Berber(99_nl): ['97_nl']
Bush Negro(97): ['122', '129']
Indisch(118): ['119']
Half-blood(110): ['135']
Mulat(132_nl): ['110_nl']
Ras(140_nl): ['124_nl', '133_nl']
Aboriginal(90): ['105', '116']
Berber(93): ['92']
Westers(146_nl): ['103_nl']
Neger(133_nl): ['140_nl', '124_nl']
Western(143): ['139']
Koppensneller(126_nl): ['137_nl']
Etniciteit(106_nl): ['140_nl']
White(144): ['95']
Black(94): ['129']


### 2. How do entries in English and Dutch glossaries correspond to each other?

In [9]:
q_2 = wm.query(
    """SELECT ?CI_en ?CI_nl
    WHERE {
    ?CI_nl skos:exactMatch ?CI_en 
    }
          
       """)

In [10]:
q_2.bindings

[{rdflib.term.Variable('CI_nl'): rdflib.term.URIRef('https://w3id.org/culco/wordsmatter#142_nl')},
 {rdflib.term.Variable('CI_nl'): rdflib.term.URIRef('https://w3id.org/culco/wordsmatter#115_nl')},
 {rdflib.term.Variable('CI_nl'): rdflib.term.URIRef('https://w3id.org/culco/wordsmatter#99_nl')},
 {rdflib.term.Variable('CI_nl'): rdflib.term.URIRef('https://w3id.org/culco/wordsmatter#116_nl')},
 {rdflib.term.Variable('CI_nl'): rdflib.term.URIRef('https://w3id.org/culco/wordsmatter#111_nl')},
 {rdflib.term.Variable('CI_nl'): rdflib.term.URIRef('https://w3id.org/culco/wordsmatter#131_nl')},
 {rdflib.term.Variable('CI_nl'): rdflib.term.URIRef('https://w3id.org/culco/wordsmatter#108_nl')},
 {rdflib.term.Variable('CI_nl'): rdflib.term.URIRef('https://w3id.org/culco/wordsmatter#114_nl')},
 {rdflib.term.Variable('CI_nl'): rdflib.term.URIRef('https://w3id.org/culco/wordsmatter#103_nl')},
 {rdflib.term.Variable('CI_nl'): rdflib.term.URIRef('https://w3id.org/culco/wordsmatter#133_nl')},
 {rdflib.te

#### How many entries are unique in both glossaries?